함께 볼 영상 

https://www.youtube.com/watch?v=GLM73CbEVaY



PDF 다운로드 

https://www.gne.go.kr/upload_data/board_data/workroom/167296838793088.pdf



In [1]:
# Ollama 모델 로드 및 테스트
from langchain_community.chat_models import ChatOllama

# model = ChatOllama(model="llama3", temperature=0)
# model = ChatOllama(model="qwen2", temperature=0)
model = ChatOllama(model="qwen2:1.5b", temperature=0)
# model = ChatOllama(model="gemma:2b", temperature=0)
# model = ChatOllama(model="gemma2", temperature=0)

response = model.invoke("겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?")

In [5]:
response

AIMessage(content='겨울철에 내한성 강한 나무에는 다음과 같은 특징이 있습니다:\n\n1. 뿌리: 내한성은 뿌리를 통해 물과 영양분을 받아들입니다. 따라서, 내한성이 강한 나무는 뿌리를 더욱 깊게 내려놓습니다.\n\n2. 잎: 내한성은 잎의 형태와 크기에 따라 달라집니다. 일반적으로 내한성이 강한 나무는 잎이 더 큰 것이나 흔히 보이는 구조를 가지고 있습니다.\n\n3. 꽃과 열매: 내한성은 꽃과 열매의 형태에도 영향을 미칩니다. 내한성이 강한 나무는 꽃이 더 높거나 꼭대기에서 피어나고, 열매가 더 큰 것이 있습니다.\n\n4. 가지: 내한성이 강한 나무는 가지가 더 긴 것과 더 넓은 구조를 가지고 있습니다.\n\n5. 성장속도: 내한성 강한 나무는 일반적으로 더 빠르게 자라나며, 이는 그들의 잎이 더 높거나 흔히 보이는 구조를 가지기 때문입니다.\n\n6. 복사력: 내한성이 강한 나무는 복사력을 가지고 있어야 합니다. 이것은 그들이 뿌리에서 물을 받아들여서 생존할 수 있는 능력에 대한 의미입니다.', response_metadata={'model': 'qwen2:1.5b', 'created_at': '2024-06-29T14:03:26.1130993Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 377158997000, 'load_duration': 373429273200, 'prompt_eval_count': 26, 'prompt_eval_duration': 15922000, 'eval_count': 321, 'eval_duration': 3711300000}, id='run-7e0b36e2-700d-4e9b-8920-1fd0094d8326-0')

In [6]:
response.content

'겨울철에 내한성 강한 나무에는 다음과 같은 특징이 있습니다:\n\n1. 뿌리: 내한성은 뿌리를 통해 물과 영양분을 받아들입니다. 따라서, 내한성이 강한 나무는 뿌리를 더욱 깊게 내려놓습니다.\n\n2. 잎: 내한성은 잎의 형태와 크기에 따라 달라집니다. 일반적으로 내한성이 강한 나무는 잎이 더 큰 것이나 흔히 보이는 구조를 가지고 있습니다.\n\n3. 꽃과 열매: 내한성은 꽃과 열매의 형태에도 영향을 미칩니다. 내한성이 강한 나무는 꽃이 더 높거나 꼭대기에서 피어나고, 열매가 더 큰 것이 있습니다.\n\n4. 가지: 내한성이 강한 나무는 가지가 더 긴 것과 더 넓은 구조를 가지고 있습니다.\n\n5. 성장속도: 내한성 강한 나무는 일반적으로 더 빠르게 자라나며, 이는 그들의 잎이 더 높거나 흔히 보이는 구조를 가지기 때문입니다.\n\n6. 복사력: 내한성이 강한 나무는 복사력을 가지고 있어야 합니다. 이것은 그들이 뿌리에서 물을 받아들여서 생존할 수 있는 능력에 대한 의미입니다.'

In [3]:
# PDF 문서 로드 및 텍스트 추출
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./수목관리업무매뉴얼.pdf", extract_images=False)
pages = loader.load()
pages[0].page_content

'경남교육 2022 - 250\n수\n목 관리 업무매뉴얼\n수목 관리 \n업무매뉴얼 쾌적한 학교 공간 재구성을 위한\n 쾌적한 학교 공간 재구성을 위한\n수목 관리\n업무매뉴얼҃թҮਭ'

In [7]:
# 문장 임베딩 및 벡터 저장소 생성
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# 문서를 문장으로 분리
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,            # 1000개의 단어 기준으로 분할을 해주고
    chunk_overlap=100,          # chunk는 100단어가 겹치도록 
)

docs = text_splitter.split_documents(pages)

# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-nli',
    model_kwargs={'device':'cpu'},  # gpu
    encode_kwargs={'normalize_embeddings':True},
)

vectorstore = Chroma.from_documents(docs, embeddings)


In [8]:
# 검색 쿼리
query = "겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?"

# 가장 유사도가 높은 문장을 하나만 추출
retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

docs = retriever.get_relevant_documents(query)
print(len(docs))
print(docs[0].page_content)
print(docs[0].metadata)

5
나) 수종별 내한성에 따른 전정 
  추위를 잘 견디어 내는 나무의 성질을 내한성이라고 한다. 생장 중인 잎이나 가는 가지 등에서 일어나는 상해(霜害)에 대한 저항성을 포함하는 경우와, 휴면 중인 가지나 엄동기
{'page': 139, 'source': './수목관리업무매뉴얼.pdf'}


c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [9]:
print(docs[1].page_content)
print(docs[1].metadata)

(1) 내한성은 동절기 중 극히 낮은 온도에 견디는 능력이다. 
(가) 내한성이 강한 수종은 한대림에서 자라는 수종으로 자작나무, 오리나무, 사
시나무, 버드나무류, 소나무, 잣나무, 전나무 등이 해당된다. 
(나) 내한성이 약한 수종은 삼나무, 편백, 금송, 히말라야시다, 배롱나무, 파라칸
타, 동백나무, 후박나무, 먼나무 등 주로 남부 지역에서 자라는 수종과 자목련, 사철나무, 가이즈까향나무, 능소화, 벽오동, 오동나무 등이다. 
(다) 내한성이 약한 수종은 수간을 볏짚이나 새끼 끈으로 싸 주고, 상열을 막기
{'page': 110, 'source': './수목관리업무매뉴얼.pdf'}


In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


# 가장 기본적인 Prompt
template = '''Answer the question based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# 페이지 내용을 하나의 문자열로 결합을 해줌
def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()} # 유사도 기반으로 가장 관련있는 내용으로 검색, 사용자 입력 
    | prompt
    | model
    | StrOutputParser()     # 생성된 답변은 문자열만 
)

# Chain 실행
query = "겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?"
rag_chain.invoke(query)


'내한성은 동절기 중 극히 낮은 온도에 견디는 능력이다. 따라서 내한성이 강한 수종은 한대림에서 자라는 수종으로 자작나무, 오리나무, 사시나무, 버드나무류, 소나무, 잣나무, 전나무 등이 해당된다.'

In [11]:
query = "겨울철에 추위에 약한 나무에는 어떤 것이 있을까요?"
rag_chain.invoke(query)

'내한성은 동절기 중 극히 낮은 온도에 견디는 능력이다. 따라서 내한성이 강한 수종은 한대림에서 자라는 수종으로 자작나무, 오리나무, 사시나무, 버드나무류, 소나무, 잣나무, 전나무 등이 해당된다. \n\n따라서 추위에 약한 나무에는 내한성이 강한 수종들이 있을 것입니다.'